<h1>SLoPP: Structural Learning of PSDDs under Partial Closed-World Assumption</h1>
<p>This notebook contains a (highly experimental and not optimized) version of SLoPP, a structural learning algorithm for PSDDs.</p>
<p>The code is using the Juice (Julia) library for Probabilistic Circuits.</p>

<h2>Imports</h2>

In [94]:
import Printf
using LogicCircuits
using ProbabilisticCircuits
using DataFrames
using BenchmarkTools
using TikzPictures
using Clustering: kmeans, nclusters, assignments
#using Statistics: mean
#using LinearAlgebra: normalize
TikzPictures.standaloneWorkaround(true)
using LogicCircuits.LoadSave: SDDElement, 
    PSDDElement, 
    save_lines,
    get_vtree2id,
    get_node2id,
    parse_psdd_file, 
    PsddHeaderLine, 
    LcHeaderLine, 
    get_nodes_level

<h2>Algorithm Parameters</h2>
<p>The user can set the number of clusters returned by the clustering algorithm and the minimum number of instances to build a cluster.</p>

In [95]:
NCLUSTERS = 2
MINCLUSTERSIZE = 50

50

<h2>Subroutines</h2>

In [96]:
function bs2arr(bs)
    y = variables(bs) # parsing the indexes of the variables on the right
    yy = (x->collect.(x)).(y)
    b = zeros(Int16,size(yy,1))
    for i = 1:size(yy,1)
        b[i] = yy[i][1]
    end
    return b
end

function lr(vtree) # returing the left and right variables of a vtree (or sub-vtree)
    if typeof(vtree.left) == PlainVtreeLeafNode && typeof(vtree.right) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = [Int16(variable(vtree.right))] 
    elseif typeof(vtree.left) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = bs2arr(vtree.right)
    else
        l = bs2arr(vtree.left)
        r = bs2arr(vtree.right)
    end    
    return [l,r]
end

# Modified version of the Juice clustering to also retrieve the assignments of the clusters 
function clustering2(data, mix_num::Int64; maxiter=200)::Vector
    n = num_examples(data)
    if mix_num == 1
        return [data]
    end
    data = Matrix(data)
    R = kmeans(data', mix_num; maxiter=maxiter)
    @assert nclusters(R) == mix_num
    a = assignments(R)
    clustered_data = Vector()
    for k in 1 : mix_num
        push!(clustered_data, DataFrame(data[findall(x -> x == k, a), :]))
    end
    return [clustered_data,a]
end

# Function to fix the col names with sub-datasets
function find_names(names_short,indexes_short,col_names)
    m=Any[]
    for i in indexes_short
        push!(m,findfirst(x->x==col_names[i], names_short))
    end
    return m
end

find_names (generic function with 1 method)

<h2>Main Function</h2>

In [97]:
function SLoPP(d,v,myrows,titles,parent=-1, element=-1, sibling=-1)
    
    node_id = rand(Int,1)[1] # Unique id for decision nodes, to be later replaced
    psdd_node_id = length(myrows) # id to identify the psdd nodes
    vtree_node_id = Int16(DICT_VTREE[v]) # id for the vtree nodes
    
    if typeof(v) == PlainVtreeLeafNode # for leav nodes a L or a T should be prepared
        
        var = Int16(variable(v)) # variable as in the vtree DEBUG check if -1
        nt = count(i->(i == true), d[:,1]) # number of true states in the first (unique) col of db d
        
        if nt == size(d,1) # all the elements true, the literal is +L
            myrow = ["L",psdd_node_id,vtree_node_id,var,parent,element,sibling]
        elseif nt == 0 # all false, the literal is -L
            myrow = ["L",psdd_node_id,vtree_node_id,-var,parent,element,sibling]           
        else # some true and some false, Top with a weight
            p = log(nt/size(d,1))
            myrow = ["T",psdd_node_id,vtree_node_id,var,p,parent,element,sibling]
        end
        
        append!(myrows,[myrow])
        
    else
        variables = [find_names(names(d),vrb,titles)  for vrb in lr(v)] # fixing names 
        
        if size(variables[1],1) == 1 # is single variable on the lft
            dd = [filter(row -> row[variables[1][1]] == i, d) for i in 0:1] # Shannon decomp
            d_left = [dd[i][:,variables[1]] for i in 1:2] # focus on left
            d_right = [dd[i][:,variables[2]] for i in 1:2] # focus on right vars
            p = [size(dd[i],1)/size(d,1) for i in 1:2] # thetas
            ps = [ ["*", "*", log(p[c])] for c = 1:2 if p[c] > 0.0]
            myrow = ["D",node_id,vtree_node_id,length(ps)]
            [append!(myrow,pps) for pps in ps]
            append!(myrow,[parent,element,sibling])
            append!(myrows,[myrow])
            for c = 1:2
                if p[c] > 0.0
                    SLoPP(d_left[c],v.left,myrows,titles,node_id,"prime",c)
                    SLoPP(d_right[c],v.right,myrows,titles,node_id,"sub",c)
                end
            end          
        else
            if size(d,1) > MINCLUSTERSIZE
                d_left = d[:,variables[1]]
                clsts2 = clustering2(d_left,NCLUSTERS)
                clsts = clsts2[1]
                assignments = clsts2[2]
                p = [size(clsts[c],1)/size(d,1) for c = 1:NCLUSTERS]
                ps = [ ["*", "*", log(p[c])] for c = 1:NCLUSTERS if p[c] > 0.0]
                myrow = ["D",node_id,vtree_node_id,length(ps)]
                [append!(myrow,pps) for pps in ps]
                append!(myrow,[parent,element,sibling])
                append!(myrows,[myrow])
                for c = 1:NCLUSTERS
                    if p[c] > 0
                        rename!(clsts[c],names(d_left))    
                        SLoPP(clsts[c],v.left,myrows,titles,node_id,"prime",c)
                        d_right = d[:,variables[2]]
                        d2 = d_right[findall(x->x==c, assignments),:]
                        SLoPP(d2,v.right,myrows,titles,node_id,"sub",c)
                    end
                end
            else
                myrow = ["D",node_id,vtree_node_id,1,"*","*",0.0,parent,element,1]
                append!(myrows,[myrow])
                SLoPP(d[:,variables[1]],v.left,myrows,titles,node_id,"prime",1)
                SLoPP(d[:,variables[2]],v.right,myrows,titles,node_id,"sub",1)
            end
        end
    end
    return myrows
end

SLoPP (generic function with 4 methods)

<h1>Replacer</h1>
<p>As the PSDD is build following a top-down procedure, the node IDs are initially generated as random ids, to be later replaced by sequential ids.</p>

In [98]:
function replacer(rows)
    dict = Dict() # learn labels for D nodes
    io = open("/Users/install/nodes.dict", "w")
    for (r,row) in enumerate(rows)
         if row[1] == "D"
             dict[row[2]] = r-1
             println(io,row[2],",\t",r-1)
         end
    end
    close(io)
    for (r,row) in enumerate(rows)
        if row[length(row)-2] != 0 # if 0 root of the circuit
            row[length(row)-2] = dict[row[length(row)-2]] # replacing parent indicator in the comments
            if row[1] == "D" 
                row[2] = dict[row[2]] # replacing right node_id
            end
            rows[r] = row
        end
    end
    rows[1][2] = dict[rows[1][2]]
end    

replacer (generic function with 1 method)

<h1>Experiments</h1>

In [ ]:
twenty_dataset_names = [
        "accidents", "ad", "baudio", "bbc", "bnetflix", "book", "c20ng", "cr52", "cwebkb",
        "dna", "jester", "kdd", "kosarek", "msnbc", "msweb", "nltcs", "plants", "pumsb_star", "tmovie", "tretail", 
        "binarized_mnist"
];


db_name = "baudio" # "plants" # "nltcs"
train_x, valid_x, test_x = twenty_datasets(db_name)

test_size = size(test_x,1)
train_size = size(train_x,1)
n_x = size(test_x,2)

println("DB:\t\t $db_name")
println("k:\t\t $NCLUSTERS")
println("d:\t\t $MINCLUSTERSIZE")
println("|X|:\t\t $n_x")
println("|D_train|:\t $train_size")
println("|D_test|:\t $test_size")


pc_cl, vtree_cl = learn_chow_liu_tree_circuit(train_x)
d = train_x[:,:] # Using fewer data for quicker tests
DICT_VTREE = get_vtree2id(vtree_cl);
pc = SLoPP(d,vtree_cl,[],names(d),0);
io = open("/Users/install/skeleton.psdd", "w")
for o in pc
    println(io,join(o[1:length(o)]," "))
end
close(io)
replacer(pc)
for (r,row) in enumerate(pc)
    if row[1] == "D"
        node_id = row[2]
        candidates = [row2 for row2 in pc if row2[length(row2)-2]==node_id]
        primes = [(c[2],c[length(c)])  for c in candidates if c[length(c)-1]=="prime"]
        subs = [(c[2],c[length(c)])  for c in candidates if c[length(c)-1]=="sub"]
        pairs = [(p,s) for (p,s) in zip(primes,subs)]
        for k in 1:length(pairs)
            #row[2+3*pairs[k][1][2]] = pairs[k][1][1]
            #row[3+3*pairs[k][2][2]] = pairs[k][2][1]
            row[2+3*k] = pairs[k][1][1] # DEBUG
            row[3+3*k] = pairs[k][2][1]
        end
        pc[r] = row
    end
end
# DEBUG
for (r,row) in enumerate(pc)
     if  occursin("*", join(row[1:length(row)]," "))
        println(row)
     end
end
io = open("/Users/install/psdd.psdd", "w")
for o in reverse(pc)
    println(io,join(o[1:length(o)-3]," "))
end
close(io)
pc_slopp = load_prob_circuit("/Users/install/psdd.psdd")
v = Vtree(num_features(train_x), :balanced)
pc_ff = fully_factorized_circuit(StructProbCircuit, v); 
pc_st = learn_circuit(train_x; maxiter = 100, verbose = false)

tot1 = 0.0
tot2 = 0.0
tot3 = 0.0
tot4 = 0.0
exc = 0
for j = 1:size(test_x,1)
    ll1 = log_likelihood_avg(pc_slopp, test_x[j:j,:])
    ll2 = log_likelihood_avg(pc_ff, test_x[j:j,:])
    ll3 = log_likelihood_avg(pc_cl, test_x[j:j,:])
    ll4 = log_likelihood_avg(pc_st, test_x[j:j,:])
    if ll1 != -Inf
        tot1 += ll1
        tot2 += ll2
        tot3 += ll3
        tot4 += ll4
    else
        exc += 1
    end
end
println("SLOPP LL:\t $tot1, $(num_nodes(pc_slopp)) nodes, $(num_parameters(pc_slopp)) par,  $exc excluded")
println("FF LL:\t\t $tot2, $(num_nodes(pc_ff)) nodes, $(num_parameters(pc_ff)) par")
println("CL LL:\t\t $tot3, $(num_nodes(pc_cl)) nodes, $(num_parameters(pc_cl)) par")
println("ST LL:\t\t $tot4, $(num_nodes(pc_st)) nodes, $(num_parameters(pc_st)) par")

DB:		 baudio
k:		 2
d:		 50
|X|:		 100
|D_train|:	 15000
|D_test|:	 3000
